#### Initial Setup

In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

#### Entity

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    source_url: str
    root_dir: Path
    unzip_dir: Path
    zip_data_path: Path

#### Configuration

In [ ]:
from src.mlops_water_potability_prediction_project.constants import *
from src.mlops_water_potability_prediction_project.utilities.helpers import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories(directories_path_list=[self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            source_url=config.source_url,
            root_dir=config.root_dir,
            unzip_dir=config.unzip_dir,
            zip_data_path=config.zip_data_path
        )

        return data_ingestion_config

#### Component

In [ ]:
import os
import urllib.request as request
import zipfile
from src.mlops_water_potability_prediction_project import logger
from src.mlops_water_potability_prediction_project.utilities.helpers import get_file_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.zip_data_path):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename=self.config.zip_data_path
            )
            logger.info(f"{filename} download! \nWith the following info: \n{headers}")
        else:
            logger.info(f"File already exists. Size : {get_file_size(Path(self.config.zip_data_path))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.zip_data_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

#### Pipeline

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e